##### Copyright 2020 Halis Sak

# Two-layer neural network implementation using Tensorflow

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True' # for avoiding problems at Mac OS

### The firm and trading characteristics data

We will work with firm and trading characteristics data that we downloaded from WRDS for this tutorial.



Read the data with pandas as before

In [4]:
df = pd.read_csv(r"/Users/halissak/Documents/SZU/courses/Financial_Econometric_Analysis/data/data_lect_materials.csv")

# Change the column names to lowercase
df.columns = [name.lower() for name in df.columns]

target = 'ret'
feats = ["rd_mve","sp","agr"]

df['year'] = [int(str(date)[:4]) for date in df.date]
ind_train = df[df.year.isin(range(1926,2005))].index # 1957 to 2004
ind_val = df[df.year.isin(range(2005,2010))].index # 2005 to 2009
ind_test = df[df.year.isin(range(2010,2017))].index # 2010 to 2016

df_train = df.loc[ind_train,:].copy().reset_index(drop=True)
df_val = df.loc[ind_val,:].copy().reset_index(drop=True)
df_test = df.loc[ind_test,:].copy().reset_index(drop=True)


### Normalize the data

Next we normalize both train and test data.

In [5]:
def normalize(series):
  return (series-series.mean(axis=0))/series.std(axis=0)

data_train = df_train[feats].apply(normalize).fillna(0).values
data_val = df_val[feats].apply(normalize).fillna(0).values
data_test = df_test[feats].apply(normalize).fillna(0).values

### Create datasets in TensorFlow
We need to use tf.data.Dataset.from_tensor_slices function to read the values from a pandas dataframe.

In [6]:
train_dataset = tf.data.Dataset.from_tensor_slices((data_train, df_train[target].values))
test_dataset = tf.data.Dataset.from_tensor_slices((data_test, df_test[target].values))

Let's check the first five elements of train_dataset.

In [9]:
for feat, targ in train_dataset.take(5):
  print ('Features: {}, Target: {}'.format(feat, targ))

Features: [0. 0. 0.], Target: 0.035088
Features: [0. 0. 0.], Target: -0.013832
Features: [0. 0. 0.], Target: -0.020474
Features: [0. 0. 0.], Target: -0.016304
Features: [0. 0. 0.], Target: 0.042945


2024-11-20 07:16:05.509124: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [23061]
	 [[{{node Placeholder/_1}}]]


## The model

### Build the model

Let's build our model. Here, we'll use a `Sequential` model with a densely connected hidden layer, and an output layer that returns a single, continuous value.

In [10]:
nfeats = len(feats)
nhid = 3 
def build_model():
  model = keras.Sequential([
    layers.Dense(nhid, activation='tanh', input_shape=[nfeats]),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.SGD(0.005)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mse'])
  return model

In [11]:
model = build_model()

We want to initiliaze model weights to random values as we have done in our lecture notes. For this we can use `.set_weights` function.

In [12]:
weights = model.weights
print(weights)

[<tf.Variable 'dense/kernel:0' shape=(3, 3) dtype=float32, numpy=
array([[ 0.38682175,  0.5173919 , -0.76048326],
       [-0.30096745, -0.22607446,  0.17635274],
       [-0.19314408,  0.32886934, -0.3316114 ]], dtype=float32)>, <tf.Variable 'dense/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>, <tf.Variable 'dense_1/kernel:0' shape=(3, 1) dtype=float32, numpy=
array([[ 0.04218173],
       [-0.25071698],
       [ 0.7282791 ]], dtype=float32)>, <tf.Variable 'dense_1/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


In [13]:
np.random.seed(12345)
w = [np.random.uniform(-0.01,0.01,size = (nfeats,nhid)),np.random.uniform(-0.01,0.01,size = nhid),np.random.uniform(-0.01,0.01,size = (nhid,1)),np.random.uniform(-0.01,0.01,size = 1)]
model.set_weights(w)

### Inspect the model

We can use the `.summary` function to print a description of the model

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 3)                 12        
                                                                 
 dense_1 (Dense)             (None, 1)                 4         
                                                                 
Total params: 16
Trainable params: 16
Non-trainable params: 0
_________________________________________________________________


The model contains two layers with 12 and 4 parameters as expected (bias units' weights are included in this sum). 

### Train the model

Let's train the model by getting only one element from the data (`batch`=1) at each step. And we want to iterate over the data only once (`epochs`=1). We chose these parameters to replicate numerical result of our feed-forward implementation.

In [15]:
model.fit(train_dataset.batch(1), epochs=1)

  227/23061 [..............................] - ETA: 5s - loss: 0.0059 - mse: 0.0059   

2024-11-20 07:16:47.176739: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [23061]
	 [[{{node Placeholder/_1}}]]
2024-11-20 07:16:47.278156: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


23061/23061 [==============================] - 5s 201us/step - loss: 0.0072 - mse: 0.0072


### Model weights
We can observe the trained model weights as follows.

In [16]:
weights = model.weights
print(weights)

[<tf.Variable 'dense/kernel:0' shape=(3, 3) dtype=float32, numpy=
array([[ 0.0077569 , -0.00429304, -0.00667997],
       [-0.00802271, -0.00033541,  0.00099836],
       [ 0.00882998,  0.00263526,  0.00461314]], dtype=float32)>, <tf.Variable 'dense/bias:0' shape=(3,) dtype=float32, numpy=array([0.00297394, 0.00479717, 0.00913184], dtype=float32)>, <tf.Variable 'dense_1/kernel:0' shape=(3, 1) dtype=float32, numpy=
array([[-0.00997811],
       [-0.00762222],
       [-0.00371891]], dtype=float32)>, <tf.Variable 'dense_1/bias:0' shape=(1,) dtype=float32, numpy=array([0.02264502], dtype=float32)>]


### Make predictions

Finally, we can make predictions on next month's returns for the test dataset:

In [17]:
test_predictions = model.predict(test_dataset.batch(1)).flatten()

 874/3192 [=======>......................] - ETA: 0s

2024-11-20 07:17:08.856450: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [3192]
	 [[{{node Placeholder/_1}}]]


3192/3192 [==============================] - 1s 166us/step


This time we used a larger batch size (`batch`=100) to make faster predictions. 

In [18]:
def R2(y, y_hat):
  R2 = 1 - np.sum((y - y_hat)**2) / np.sum(y**2)
  return R2

R2_Val = R2(df_test[target].values,test_predictions)
print(R2_Val)

0.008362189215720184


The R2 value is very close to what we computed in our implementation (0.00862). 